# Multi-agent SIPP

In [1]:
import networkx as nx
from matplotlib import pyplot as plt
import numpy as np

import sys
sys.path.append('../')

sys.path.append('../../')

from panav.util import interpolate_positions

from panav.SIPP import compute_safe_intervals, plan_to_transitions,SIPP

from matplotlib.animation import FuncAnimation

from IPython.display import HTML, display


from panav.viz import animate_MAPF_R

%load_ext autoreload
%autoreload 2

In [7]:
from panav.MAPFR_conflict import MAPFR_conflict

from panav.PBS.HighLevelSearchTree import PriorityTree, SearchNodeContainer
from panav.util import unique_tx
import itertools
from queue import PriorityQueue
from copy import deepcopy

def flowtime(plan):
    return np.sum([wp[-1][1] for wp in plan])
def makespan(plan):
    return np.max([wp[-1][1] for wp in plan])


def PBS_SIPP(G,node_locs,starts,goals,\
             vmax,bloating_r,\
        max_iter = 200,metric = 'flowtime',search_type = 'depth_first'\
        ):
    '''
        The MAMP algorithm using PBS as the high-level search and tube-based SAMP as the low-level search.
        
        Inputs: 
            G: The path planning environment.
            
            node_locs: A dict {s:loc[s] for s in G}, storing the location of each node.
            
            starts, goals: The start and goal nodes.
            
            vmax,bloating_r: SIPP parameters.
                
            max_iter: the maximal iteration of PBS mainloop before force exit and return solution not found.

            metric: the metric used in high-level search, can be either 'flowtime' or 'makespan'.

            search_type: the branching style used in high-level search, can be either 'depth_first'(fast) or 'best_first'(slow).
        
        Output: a multi-agent plan in the form of [(s_i,t_i)]
    '''
    agents = set(np.arange(len(starts)))
    if metric == 'flowtime':
        metric = flowtime
    elif metric == 'makespan':
        metric = makespan
    else:
        print('Metric {} is not supported. Please input "flowtime" or "makespan".'.format(metric))

    # Initialize the agents' plans
    plan0 = []
    for agent in agents:
        start,goal = starts[agent],goals[agent]
        plan = SIPP(G,node_locs,start,goal,[],vmax,bloating_r)
        plan0.append(plan)
    
    print(plan0)
    
    cost0 = metric(plan0)

    PT = PriorityTree()
    ROOT = PT.add_node(None,plan0,cost0,ordering=[]) # Adding the root node. 

    OPEN = SearchNodeContainer(search_type)
    OPEN.push(cost0,ROOT)
    
    # Enter the main PBS loop.
    count = 0
    while not OPEN.empty() and count<=max_iter:
        count+=1 

        cost, parent_node = OPEN.pop()
        solution = PT.get_solution(parent_node)

        conflict = MAPFR_conflict(solution,node_locs,bloating_r) # Look for the first conflict.
        if conflict is None:
            return solution, cost
        else:
            a1,a2 = conflict['agents'] # Get the two agents involved in the conflict.

        prev_ordering = PT.get_ordering(parent_node)
        new_PT_nodes = PriorityQueue()

        # Compute new PT nodes
        for (j,i) in [(a1,a2),(a2,a1)]:
            new_plan = deepcopy(solution)
            new_order = [(j,i)] 
            if (i,j) in prev_ordering or len(list(nx.simple_cycles(nx.DiGraph(prev_ordering+new_order))))>0: 
                    print('Skipping ij',(j,i),'prev_ordering',prev_ordering)
                    continue # Do not add (j,i) to the partial ordering if it introduces a cycle.
            curr_ordering = prev_ordering+new_order

            sorted_agents  = list(nx.topological_sort(nx.DiGraph(curr_ordering))) # Get all the agents with lower orderings than i.

            idx_i = np.where(np.array(sorted_agents)==i)[0][0]

            agents_to_avoid = [a for a in sorted_agents[:idx_i]]

            success_update = True
            for k in range(idx_i, len(sorted_agents)):
                agent_to_update = sorted_agents[k]
                
                # Update the plan for agent_to_update
                start, goal = starts[agent_to_update],goals[agent_to_update]
                obs_transitions = itertools.chain.from_iterable(\
                                    [plan_to_transitions(new_plan[av]) \
                                     for av in agents_to_avoid])

                result = SIPP(G,node_locs,start,goal,obs_transitions,vmax,bloating_r)

                if result is not None:
                    new_plan[agent_to_update] = result
                    agents_to_avoid.append(agent_to_update)
                else:
                    success_update = False 
                    break
                    
            if success_update:
                cost = metric(new_plan) 

                neg_of_cost = -cost
                new_node = PT.add_node(parent_node, new_plan, cost,new_order)
                new_PT_nodes.put((neg_of_cost, new_node)) 
                # This is to ensure the PT node with higher cost will be added first to OPEN.


        # Put the (at most two) new PT nodes onto OPEN in non-increasing order of the cost.
        while not new_PT_nodes.empty():
            neg_of_cost, PT_node = new_PT_nodes.get()
            cost = -neg_of_cost
            OPEN.push(cost, PT_node)

    print('Total iterations = ',count,'OPEN empty?',OPEN.empty())
    return None

In [3]:
node_locs = np.array([[0,1],
                      [0,-1],
                      [1,0],
                      [2,0],
                      [3,0],
                      [4,0],
                      [5,1],
                      [5,-1]])
N = len(node_locs)

G = nx.DiGraph()
G.add_nodes_from(range(N))
edges = [(0,2),
          (1,2),
          (2,3),
          (3,4),
          (4,5),
          (5,6),
          (5,7)]

G.add_edges_from(edges)
G.add_edges_from(e[::-1] for e in edges)

# We manually add the bi-directional edges here, because
# using nx.to_directed(a undirected graph) causes issue when accessing edge values.

In [9]:
starts = [0,7]
goals = [6,1]

v_max = 1.0
bloating_r = 0.2

nx.set_edge_attributes(G,
                       {e:np.linalg.norm(node_locs[e[0]]-node_locs[e[1]])/v_max\
                          for e in G.edges},
                       'weight')

In [10]:
joint_plan,ft = PBS_SIPP(G,node_locs,starts,goals,v_max,bloating_r)

[[(0, 0), (2, 1.4142135623730951), (3, 2.414213562373095), (4, 3.414213562373095), (5, 4.414213562373095), (6, 5.82842712474619)], [(7, 0), (5, 1.4142135623730951), (4, 2.414213562373095), (3, 3.414213562373095), (2, 4.414213562373095), (1, 5.82842712474619)]]
Skipping ij (0, 1) prev_ordering [(1, 0)]
Skipping ij (0, 1) prev_ordering [(1, 0), (1, 0)]
Skipping ij (0, 1) prev_ordering [(1, 0), (1, 0), (1, 0)]
Skipping ij (0, 1) prev_ordering [(1, 0), (1, 0), (1, 0), (1, 0)]
Skipping ij (0, 1) prev_ordering [(1, 0), (1, 0), (1, 0), (1, 0), (1, 0)]
Skipping ij (0, 1) prev_ordering [(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0)]
Skipping ij (0, 1) prev_ordering [(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0)]
Skipping ij (0, 1) prev_ordering [(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0)]
Skipping ij (0, 1) prev_ordering [(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0)]
Skipping ij (0, 1) prev_ordering [(1, 0), (1, 0), (1, 0), (1, 0), (1, 0),

TypeError: cannot unpack non-iterable NoneType object

In [6]:
joint_plan

NameError: name 'joint_plan' is not defined

In [ ]:
dt = 0.05
anim = animate_MAPF_R(G,node_locs,[],joint_plan,dt,bloating_r)
display(HTML(anim.to_jshtml()))
plt.close() 